
## MLflow Integration for Model Training and Tracking

In this notebook, we're integrating MLflow into a machine learning workflow to track and manage experiments effectively. We're focusing on a text classification task using the DistilBert model, emphasizing the importance of experiment tracking, model management, and operational efficiency - core themes of our course.


### Objective:

- Dynamically set up and log parameters to MLflow
- Understand the purpose and application of each step in the context of MLflow and MLOps principles


### Environment Setup

Ensure all necessary libraries are installed and imported for our workflow.



### Imports

Import the necessary libraries, focusing on MLflow for tracking, PyTorch for model training, and Transformers for our NLP model.

In [2]:
# !pip install datasets
import os
import mlflow
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW


/Users/sebastiensime/Documents/mlops-introduction/mlflow_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration Parameters as an Object

By defining parameters as a dictionary, we can easily iterate through them when logging to MLflow. This method streamlines the process and adheres to best practices in code maintainability and scalability.


In [3]:
params = {
    'model_name': 'distilbert-base-cased',
    'learning_rate': 5e-5,
    'batch_size': 16,
    'num_epochs': 1,
    'dataset_name': 'ag_news',
    'task_name': 'sequence_classification',
    'log_steps': 100,
    'max_seq_length': 128,
    'output_dir': 'models/distilbert-base-uncased-ag_news',
}


### MLflow Setup

Setting up MLflow is crucial for tracking our experiments, parameters, and results, allowing us to manage and compare different runs effectively - a practice that aligns with the MLOps goal of systematic and efficient model management.

In [4]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"{params['task_name']}")

2024/06/13 20:47:48 INFO mlflow.tracking.fluent: Experiment with name 'sequence_classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/sebastiensime/Documents/mlops-introduction/artifact_store/2', creation_time=1718304468092, experiment_id='2', last_update_time=1718304468092, lifecycle_stage='active', name='sequence_classification', tags={}>

### Load and Preprocess Dataset

We're using a well-known NLP dataset to ensure reproducibility and comparability. The preprocessing step is crucial for converting raw text into a format that our model can understand, highlighting the importance of data preparation in the ML pipeline.

In [6]:
# Load and preprocess dataset
dataset = load_dataset(params['dataset_name'])#, params['task_name'])
tokenizer = DistilBertTokenizer.from_pretrained(params['model_name'])

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=params['max_seq_length'])


train_dataset = dataset["train"].shuffle().select(range(20_000)).map(tokenize, batched=True)
test_dataset = dataset["test"].shuffle().select(range(2_000)).map(tokenize, batched=True)

# Set format for PyTorch and create data loaders
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

# get the labels
labels = dataset["train"].features['label'].names

Generating test split: 100%|██████████| 7600/7600 [00:00<00:00, 1939798.60 examples/s]
/Users/sebastiensime/Documents/mlops-introduction/mlflow_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:00<00:00, 4633.11 examples/s]



### Model Initialization

Initializing the model is a foundational step, showcasing the practical application of a pre-trained NLP model for a specific task - reflecting the course's focus on real-world applicability of machine learning models.

In [7]:
model = DistilBertForSequenceClassification.from_pretrained(params['model_name'], 
                                                            num_labels=len(labels))
model.config.id2label = {i: label for i, label in enumerate(labels)}
params['id2label'] = model.config.id2label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Optimizer Setup

Choosing the right optimizer and learning rate is vital for effective model training. It demonstrates the importance of hyperparameter tuning, a key concept in achieving optimal model performance.

In [8]:
optimizer = AdamW(model.parameters(), lr=params['learning_rate'])

/Users/sebastiensime/Documents/mlops-introduction/mlflow_env/lib/python3.12/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Evaluation Function

Evaluating the model on a separate test set helps us understand its performance on unseen data, highlighting the concept of generalization which is crucial for real-world applications.

In [9]:
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

            # Forward pass, calculate logit predictions
            outputs = model(inputs, attention_mask=masks)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate Evaluation Metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


### Training Loop

The training loop is where the actual model training happens. Logging metrics and parameters at each step is crucial for tracking the model's progress, understanding its behavior, and making informed decisions - core aspects of the MLOps lifecycle.

In [10]:
# Start MLflow Run
with mlflow.start_run(run_name=f"{params['model_name']}-{params['dataset_name']}") as run:

    # Log all parameters at once
    mlflow.log_params(params)

    with tqdm(total=params['num_epochs'] * len(train_loader), desc=f"Epoch [1/{params['num_epochs']}] - (Loss: N/A) - Steps") as pbar:
        for epoch in range(params['num_epochs']):
            running_loss = 0.0
            for i, batch in enumerate(train_loader, 0):
                inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

                optimizer.zero_grad()
                outputs = model(inputs, attention_mask=masks, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i and i % params['log_steps'] == 0:
                    avg_loss = running_loss / params['log_steps']

                    pbar.set_description(f"Epoch [{epoch + 1}/{params['num_epochs']}] - (Loss: {avg_loss:.3f}) - Steps")
                    mlflow.log_metric("loss", avg_loss, step=epoch * len(train_loader) + i)

                    running_loss = 0.0
                pbar.update(1)

            # Evaluate Model
            accuracy, precision, recall, f1 = evaluate_model(model, test_loader, device)
            print(f"Epoch {epoch + 1} Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

            # Log metrics to MLflow
            mlflow.log_metrics({'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}, step=epoch)


    # Log model to MLflow through built-in PyTorch method
    # mlflow.pytorch.log_model(model, "model")

    # Log model to MLflow through custom method
    os.makedirs(params['output_dir'], exist_ok=True)
    model.save_pretrained(params['output_dir'])
    tokenizer.save_pretrained(params['output_dir'])

    mlflow.log_artifacts(params['output_dir'], artifact_path="model")

    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri, "agnews-transformer")

print('Finished Training')

Epoch [1/1] - (Loss: 0.241) - Steps: 100%|██████████| 1250/1250 [16:24<00:00,  1.27it/s]

Epoch 1 Metrics: Accuracy: 0.9085, Precision: 0.9107, Recall: 0.9094, F1: 0.9081



Successfully registered model 'agnews-transformer'.
2024/06/13 21:11:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: agnews-transformer, version 1
Created version '1' of model 'agnews-transformer'.


Finished Training
